In [ ]:
Deep Learning Model to Predict Comsumption

Build deep learning model to predict consumption using TensorFlow (keras)
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
# import tensorflow_transform as tft
print(tf.__version__)
FILE = 'recs2009_public.csv'
df= pd.read_csv(FILE)
df_dummies =  pd.get_dummies(df)
# X and y for supervised learning
X = df_dummies.drop(['KWH'],axis = 1)
y = df_dummies['KWH']
# Split train and test data
from sklearn.model_selection import train_test_split
X_train_pre, X_test_pre, y_train, y_test = train_test_split(X, y)
# Preprocessing & Normalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_pre)
# Train and Test data before feature selection
X_train_prefs = scaler.transform(X_train_pre)
X_test_prefs = scaler.transform(X_test_pre)
#Assign train and test data
X_train = X_train_prefs
X_test = X_test_prefs
N  = len(X_train)
print(N)
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=X_train.shape[1]),
        layers.Dense(64, activation='relu',activity_regularizer = 'l2'),
        layers.Dense(32, activation='relu',activity_regularizer = 'l2'),
        layers.Dense(1)])
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    
    model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae', 'mse'])
    return model
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('.', end='')
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [KWH]')
    plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
#   plt.ylim([0,5])
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$KWH^2$]')
    plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
#   plt.ylim([0,20])
    plt.legend()
    plt.show()
model = build_model()
model.summary()
EPOCHS = 1000

history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split = 0.2,
                    verbose=0,callbacks=[PrintDot()])

plot_history(history)
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()
loss, mae, mse = model.evaluate(X_test, y_test, verbose=0)
print("Testing set Mean Abs Error: {:5.2f} KWH".format(mae))
test_predictions = model.predict(X_test).flatten()
from sklearn.metrics import r2_score
r2 = r2_score(y_test, test_predictions)

plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [KWH]')
plt.ylabel('Predictions [KWH]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
plt.text(25000,120000,'R2 = {:3.3f}'.format(r2))
_ = plt.plot([0, 150000], [0, 150000])
error = test_predictions - y_test
plt.hist(error, bins = 20)
plt.xlabel("Prediction Error [KWH]")
_ = plt.ylabel("Count")
